# Distinct dev sets

In [5]:
data_within_path = '../argmining19-same-side-classification/data/same-side-classification/within-topic/{}.csv'

4381 pairs
* overlapping (both arguments of each test pair have been seen during training)
* distinct
* random

In [1]:
import pandas as pd
import numpy as np
import pickle
import csv
from nltk.tokenize import sent_tokenize, word_tokenize

data_cross_path = '../argmining19-same-side-classification/data/same-side-classification/cross-topic/{}.csv'
data_within_path = '../argmining19-same-side-classification/data/same-side-classification/within-topic/{}.csv'

within_train_df = pd.read_csv(data_within_path.format('training'),
                                 quotechar='"',
                                 quoting=csv.QUOTE_ALL,
                                 encoding='utf-8',
                                 escapechar='\\',
                                 doublequote=False,
                                 index_col='id')

In [2]:
within_train_df.head(2)

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic
id,,,,,,,
85249,"gay marriage devalues marriage, frequency of o...",d2f4b1cd-2019-04-17T11:47:27Z-00063-000,being unaccustomed to gay marriage is no argument,d2f4b1cd-2019-04-17T11:47:27Z-00063-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage"
2607,accepted. pro may extend their arguments to th...,2a0d32eb-2019-04-18T11:46:44Z-00004-000,"i""m pro-life. just think about it, your murder...",2a0d32eb-2019-04-18T11:46:44Z-00004-000,2a0d32eb-2019-04-18T11:46:44Z,False,abortion (pro life)


In [3]:
print(within_train_df.iloc[3]["argument1"])
print("---")
print(within_train_df.iloc[3]["argument2"])

abortion should be banned nationally unless it is necessary to save the life of the mother. i added the "except to save the life of the mother" part really more to appease those who would say that it should be an exception, though really the only situation that i can think of would be an ectopic pregnancy. to solve that, it does kill the baby, but technically it isn't an abortion, just the removal of a tube. mainly, i want to make it clear that there would be no exceptions for rape/incest, because the issue hear isn't taking responsibility for ones actions; rather it is out of respect for the sanctity of human life.
---
abortion should be banned nationally unless it is necessary to save the life of the mother. im sorry i was not thinking when i signed this to debate you i toatally agree with you on what you are sayin im just glad there are people out there like you because thats exactly what this country needs.


In [4]:
cluster_dict = {} # arg_id to cluster_id
cluster_total = {} # cluster_id to counts arg_id pairs
cluster_unique = {} # cluster_id to set of arg_ids
index2cluster = {}
cluster2arg = {}
both_seen = []
delete_candidate = []

within_train_df = within_train_df.sort_values(by=['argument1', 'argument2'])

def build_index(d):
    index = {}
    for arg, cluster_id in d.items():
        if cluster_id not in index:
            index[cluster_id] = set()
        index[cluster_id].add(arg)
    return index

for index, row in within_train_df.iterrows():
    
    cluster_id = None
    
    if row['argument1'] in cluster_dict and row['argument2'] in cluster_dict:
        both_seen_candidate = True
    else:
        both_seen_candidate = False
    both_seen.append(both_seen_candidate)
    
    if row['argument1'] in cluster_dict:
        cluster_id = cluster_dict[row['argument1']]
        if row['argument2'] in cluster_dict and cluster_dict[row['argument1']] != cluster_dict[row['argument2']]:
            
            # skip some data instances from the original set to receive disconnected partial graphs
            delete_candidate.append(True)
            continue
            
            #print('Merge cluster %d to %d' % (cluster_dict[row['argument2']], cluster_dict[row['argument1']]))
            #for arg in cluster_unique[cluster_dict[row['argument2']]]:
            #    cluster_dict[arg] = cluster_id
            #cluster_unique = build_index(cluster_dict)            
    elif row['argument2'] in cluster_dict:
        cluster_id = cluster_dict[row['argument2']]       
    else:
        cluster_id = len(cluster_unique)
        cluster_unique[cluster_id] = set()
        cluster_total[cluster_id] = 0
        
    cluster_dict[row['argument1']] = cluster_id
    cluster_dict[row['argument2']] = cluster_id
    
    cluster_unique[cluster_id].add(row['argument1'])
    cluster_unique[cluster_id].add(row['argument2'])
        
    cluster_total[cluster_id] += 1
    
    delete_candidate.append(False)

In [5]:
within_train_df['del'] = delete_candidate
within_train_df['both_seen'] = both_seen

In [6]:
index2cluster = []
selected_df = within_train_df[within_train_df['del']==False].copy()
for index, row in selected_df.iterrows():
    index2cluster.append(cluster_dict[row['argument1']])

In [7]:
selected_df['cluster'] = index2cluster

In [8]:
df_list = []
for cluster_id in cluster_total.keys():
    df_list.append({
        'cluster_id' : cluster_id,
        'total' : cluster_total[cluster_id],
        'unique' : len(cluster_unique[cluster_id]),
        'ratio' : cluster_total[cluster_id] / len(cluster_unique[cluster_id])
    })
df = pd.DataFrame(df_list).sort_values(by=['total'], ascending = False)
df

,cluster_id,total,unique,ratio
35,35,21226,251,84.565737
147,147,11678,186,62.784946
953,953,3417,100,34.170000
128,128,1419,66,21.500000
540,540,1271,62,20.500000
...,...,...,...,...
2027,2027,1,2,0.500000
2026,2026,1,2,0.500000
2021,2021,1,2,0.500000
2016,2016,1,2,0.500000


In [9]:
cluster_total[4]

34

In [10]:
print(df.sum(0)[1], df.sum(0)[1] * 0.1)

62358.0 6235.8


In [11]:
# Die nach Größe sortierten Cluster kann man ab einem bestimmten i aufteilen 
# in gerade/ungerade cluster id und so die etwa gleich großen distinct/overlap
# sets erzeugen
i = 749
df.iloc[:i, :].sum(0).tolist()

[740314.0, 53596.0, 6590.0, 2000.5488701341862]

In [12]:
df.iloc[i::2, :].sum(0).tolist()

[1103447.0, 4384.0, 3490.0, 942.8071428571402]

In [13]:
df.iloc[i+1::2, :].sum(0).tolist()

[1105045.0, 4378.0, 3494.0, 939.9499999999971]

In [14]:
# split distinct datasets
train_df = pd.concat([df.iloc[:i, :], df.iloc[i::2, :]])
test_df = df.iloc[i+1::2, :]

In [15]:
print(train_df.sum(0).tolist())
print(test_df.sum(0).tolist())

[1843761.0, 57980.0, 10080.0, 2943.3560129913135]
[1105045.0, 4378.0, 3494.0, 939.9499999999971]


In [16]:
distinct_train_df = selected_df[selected_df['cluster'].isin(train_df['cluster_id'])]
distinct_dev_df = selected_df[selected_df['cluster'].isin(test_df['cluster_id'])]

### split sets into argument pairs with complete overlap

In [17]:
current_cluster = None
both_seen_selection = []
selected_args = set()
for index, row in selected_df.iterrows():
    if row['both_seen']:        
        if not (row['argument1'] in selected_args or row['argument2'] in selected_args):
            both_seen_selection.append(True)
            selected_args.add(row['argument1'])
            selected_args.add(row['argument2'])
        else:
            both_seen_selection.append(False)
    else:
        both_seen_selection.append(False)

In [18]:
selected_df['overlap'] = both_seen_selection
sum(both_seen_selection)

4381

In [19]:
overlap_train_df = selected_df[selected_df['overlap'] == False]
overlap_dev_df = selected_df[selected_df['overlap'] == True]

In [53]:
from sklearn.utils import shuffle
rnd_within = shuffle(within_train_df, random_state=17)
rnd_dev_df = rnd_within[:4381]
rnd_train_df = rnd_within[4381:selected_df.shape[0]]
check_splits(rnd_train_df, rnd_dev_df)

Train, dev, both, one, distinct: 57977 4381 4274 95 12


### Write to disk

In [54]:
# distinct_train_df = shuffle(distinct_train_df)
# distinct_test_df = shuffle(distinct_test_df)
# overlap_train_df = shuffle(overlap_train_df)
# overlap_dev_df = shuffle(overlap_dev_df)

with open("data/distinct_sets/within-v2.pkl", "wb") as f:
    pickle.dump(distinct_train_df, f)
    pickle.dump(distinct_dev_df, f)
    pickle.dump(overlap_train_df, f)
    pickle.dump(overlap_dev_df, f)
    pickle.dump(rnd_train_df, f)
    pickle.dump(rnd_dev_df, f)

In [22]:
### check splits

In [23]:
print(selected_df['is_same_side'].sum())
print(selected_df.shape)

33380
(62358, 11)


In [24]:
print(distinct_train_df['is_same_side'].sum() + distinct_dev_df['is_same_side'].sum())
print(distinct_train_df.shape)

33380
(57980, 10)


In [25]:
print(overlap_train_df['is_same_side'].sum() + overlap_dev_df['is_same_side'].sum())
print(overlap_train_df.shape)

33380
(57977, 11)


In [23]:
print(rnd_train_df['is_same_side'].sum() + rnd_dev_df['is_same_side'].sum())
print(rnd_train_df.shape)

33276
(57977, 9)


In [24]:
def check_splits(train, dev):
    all_args = set()
    for index, row in train.iterrows():
        all_args.add(row['argument1'])
        all_args.add(row['argument2'])
    both = 0
    one = 0
    distinct = 0
    for index, row in dev.iterrows():
        if row['argument1'] in all_args and row['argument2'] in all_args:
            both +=1
        elif row['argument1'] in all_args or row['argument2'] in all_args:
            one += 1
        else:
            distinct += 1
    print("Train, dev, both, one, distinct:", train.shape[0], dev.shape[0], both, one, distinct)            

In [25]:
check_splits(rnd_train_df, rnd_dev_df)

Train, dev, both, one, distinct: 57977 4381 4274 95 12


In [56]:
check_splits(overlap_train_df, overlap_dev_df)

Train, dev, both, one, distinct: 57977 4381 4381 0 0


In [57]:
check_splits(distinct_train_df, distinct_dev_df)

Train, dev, both, one, distinct: 57980 4378 0 0 4378


In [1]:
# check saved dataset
import pickle
with open("data/distinct_sets/within-v2.pkl", "rb") as f:
    distinct_train_df = pickle.load(f)
    distinct_dev_df = pickle.load(f)
    overlap_train_df = pickle.load(f)
    overlap_dev_df = pickle.load(f)
    rnd_train_df = pickle.load(f)
    rnd_dev_df = pickle.load(f)

In [ ]:
# test for overlap fo train and dev / duplicates in train etc.
# ... apparently there are some duplicates in the original dataset, and some arg1-arg2 arg2-arg1 permutations
pair2index = {}
for index, row in within_train_df.iterrows():
    if row['argument1'] == row['argument2']:
        print(index)
    comb = row['argument1'] + row['argument2']
    if comb in pair2index:
        print("match")
    if row['argument2'] + row['argument1'] in pair2index:
        print("march rev")
    pair2index[comb] = index

In [19]:
# hewever, these duplicates are only few
n = 0
for index, row in rnd_dev_df.iterrows():
    comb = row['argument1'] + row['argument1']
    if comb in pair2index:
        n += 1
print(n)

8
